# TensorRider Algorithm
Building regression model with TensorFlow's native APIs.

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf

BATCH_SIZE = 256
EPOCHS = 300

# def imgPreprocess(rgb):
#     r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
#     # tgt = 0.2989 * r + 0.5870 * g + 0.1140 * b
#     tgt = 0.5 * r + 0.25 * g + 0.25 * b
#     return tgt

# Prepare

In [2]:
def decode(serialized_example):
    
    features = tf.parse_single_example(
            serialized_example,
            features={
                    'image': tf.FixedLenFeature([], tf.string),
                    'label': tf.FixedLenFeature([], tf.int64),
            })

    image = tf.decode_raw(features['image'], tf.uint8)
    image.set_shape((4800))

    label = tf.cast(features['label'], tf.int32)
    
    return image, label


def augment(image, label):
    
    return image, label


def normalize(image, label):
    image = tf.cast(image, tf.float32) * (1. / 255) - 0.5
    return image, label

def inputs(file, batch_size, num_epochs):

    filename = file + '.tfrecords'

    with tf.name_scope('input'):
        
        dataset = tf.data.TFRecordDataset(filename)

        dataset = dataset.map(decode)
        dataset = dataset.map(augment)
        dataset = dataset.map(normalize)

        dataset = dataset.shuffle(1000 + 3 * batch_size)

        dataset = dataset.repeat(num_epochs)
        dataset = dataset.batch(batch_size)

        iterator = dataset.make_one_shot_iterator()
        
    return iterator.get_next()

In [6]:
x,y = inputs('train', 16, 10)
print(x)
sess.run(x)

Tensor("IteratorGetNext_2:0", shape=(?, 4800), dtype=float32)


array([[-0.11960781, -0.13529411, -0.14705881, ...,  0.20980394,
         0.22941178,  0.20588237],
       [-0.01764703, -0.04901958,  0.00196081, ...,  0.08039218,
         0.10392159,  0.08823532],
       [-0.0333333 , -0.11568624, -0.0960784 , ...,  0.16274512,
         0.18235296,  0.15882355],
       ...,
       [-0.19019607, -0.25686273, -0.22549018, ..., -0.19411764,
        -0.15882352, -0.17843136],
       [ 0.02941179, -0.04509801, -0.06078428, ..., -0.23725489,
        -0.17058823, -0.1078431 ],
       [ 0.25686282,  0.23333335,  0.2725491 , ..., -0.39803922,
        -0.35490197, -0.33137256]], dtype=float32)

# Define the Dense Layers

In [4]:
x = tf.placeholder(tf.float32, [None, 4800])
W = tf.Variable(tf.zeros([4800, 1]))
b = tf.Variable(tf.zeros([1]))
y = tf.matmul(x, W) + b

y_ = tf.placeholder(tf.float32, [None, 1])

msqr = tf.reduce_mean(tf.losses.mean_squared_error(y, y_))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(msqr)

# Train

In [5]:
sess = tf.InteractiveSession()
for _ in range(EPOCHS):
    batch_xs, batch_ys = inputs(file='train', batch_size=BATCH_SIZE, num_epochs=EPOCHS)
    batch_xs, batch_ys = sess.run([batch_xs, batch_ys])
    _, error = sess.run(train_step, feed_dict={x: batch_xs, y_:batch_ys})
    print(_,error)

ValueError: Cannot feed value of shape (256, 14400) for Tensor 'Placeholder:0', which has shape '(?, 4800)'